# YOLOv8

In questo notebook, a partire da un dataset precedentemente processato (https://www.kaggle.com/code/angelobarletta/yolo-patches) si realizza l'addestramento della rete YOLOv8. Dopodiché il modello ottenuto e validato viene testato sul set di immagini di test (TestImages)

In [ ]:
pip install ultralytics

In [ ]:
import os
import numpy as np
import pandas as pd
import requests
from io import BytesIO
import cv2
import matplotlib.pyplot as plt
import imageio
from ultralytics import YOLO
from PIL import Image,ImageDraw
import zipfile
import os
from IPython.display import FileLink
import shutil
import copy
from tqdm import tqdm
from __future__ import division
import scipy.optimize
import numpy as np

## Helper functions

In [ ]:
# funzione per il download dell'output
def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)


# funzione per mostrare l'immagine con i relativi box
def show_image_with_boxes(image_path, annotation_file):
    # Carica l'immagine come array NumPy
    image_array = imageio.v2.imread(image_path)
    
    # Converti l'array NumPy in un oggetto Image
    image = Image.fromarray(image_array)
    
    # Crea un oggetto ImageDraw per disegnare i bounding box sull'immagine
    draw = ImageDraw.Draw(image)
    
    # Dizionario che mappa classi a colori
    class_colors = {
        48: "yellow",
        5: "green",
    }
    default_color = 'red'

    # Leggi le annotazioni dal file di testo
    with open(annotation_file, 'r') as f:
        for line in f:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            
            # Calcola le coordinate del bounding box in formato assoluto
            image_width, image_height = image.size
            x1 = int((x_center - width / 2) * image_width)
            y1 = int((y_center - height / 2) * image_height)
            x2 = int((x_center + width / 2) * image_width)
            y2 = int((y_center + height / 2) * image_height)
            
            color = class_colors.get(int(class_id), default_color)

            # Disegna il bounding box sull'immagine
            draw.rectangle([x1, y1, x2, y2], outline=color, width=3)
            draw.text((x1, y1), f"Class: {int(class_id)}", fill=color)

    # Mostra l'immagine con i bounding box
    plt.figure()
    plt.imshow(image)
    
    return image


# funzione per mostrare il grafico della loss di train e validation
def print_plot(type_loss):
    
    # Leggi il CSV
    df = pd.read_csv("/kaggle/working/runs/detect/train/results.csv")

    df.columns = df.columns.str.strip()   #Elimina spazi prima degli indici
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)   #elimina spazi prima dei valori

    # Estrai le colonne necessarie
    epochs = df['epoch']
    train_loss = np.float64(df['train/'+type_loss])
    val_loss = np.float64(df['val/'+type_loss])

    # Crea il grafico
    plt.figure(figsize=(6, 6))
    plt.plot(epochs, train_loss, label=type_loss+' (Train)')
    plt.plot(epochs, val_loss, label=type_loss+' (Val)')

    # Aggiungi etichette e titolo
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Over Epochs')
    plt.legend()

    # Mostra il grafico
    plt.show()

    
# funzione per mostrare il grafico di mAP di train e validation
def print_metric(type_metric):
    
    # Leggi il CSV
    df = pd.read_csv("/kaggle/working/runs/detect/train/results.csv")

    df.columns = df.columns.str.strip()   #Elimina spazi prima degli indici
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)   #elimina spazi prima dei valori

    # Estrai le colonne necessarie
    epochs = df['epoch']
    metric = np.float64(df['metrics/'+type_metric])

    # Crea il grafico
    plt.figure(figsize=(6, 6))
    plt.plot(epochs, metric, label=type_metric)

    # Aggiungi etichette e titolo
    plt.xlabel('Epoch')
    plt.ylabel(type_metric)
    plt.title(type_metric+' over epochs')
    plt.legend()

    # Mostra il grafico
    plt.show()
    
    

# funzione per il match dei box reali e predetti
def bbox_iou(boxA, boxB):
    
  # Determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interW = xB - xA
    interH = yB - yA

    # reject non-overlapping boxes
    if interW <=0 or interH <=0 :
        return -1.0

    interArea = interW * interH
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou



def match_bboxes(bbox_gt, bbox_pred, IOU_THRESH=0.5):
    # Given sets of true and predicted bounding-boxes, determine the best possible match.
    n_true = bbox_gt.shape[0]
    n_pred = bbox_pred.shape[0]
    MAX_DIST = 1.0
    MIN_IOU = 0.0

    # NUM_GT x NUM_PRED
    iou_matrix = np.zeros((n_true, n_pred))
    for i in range(n_true):
        for j in range(n_pred):
            iou_matrix[i, j] = bbox_iou(bbox_gt[i,1:], bbox_pred[j,1:])
   
    # call the Hungarian matching
    idxs_true, idxs_pred = scipy.optimize.linear_sum_assignment(1 - iou_matrix)

    ious_actual = iou_matrix[idxs_true, idxs_pred]
    sel_valid = (ious_actual > IOU_THRESH)
    label = sel_valid.astype(int)

    return idxs_true, idxs_pred, ious_actual, label 



# funzione per convertire le coordinate dal formato [x_center, y_center, width, height] a [x_min, y_min, x_max, y_max]
def change_coordinate_format(coordinate_list):
    for i,coordinate in enumerate(coordinate_list):
        x_center = coordinate[1]
        y_center = coordinate[2]
        width = coordinate[3]
        heigth = coordinate[4]
        coordinate[1] = x_center - width/2    # x_min
        coordinate[2] = y_center - heigth/2   # y_min
        coordinate[3] = x_center + width/2    # x_max
        coordinate[4] = y_center + heigth/2   # y_max


        
# calcolo della precision per un'immagine
def calculate_precision(tp, fp):
    # Calcola la precisione
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    return precision



# calcolo della recall per un'immagine
def calculate_recall(tp, fn):
    # Calcola il recall
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    return recall



# calcolo dell'f1 score per un'immagine
def calculate_f1_score(precision, recall):
    # Calcola l'F1 Score
    f1_score = 2 * ((precision * recall) / (precision + recall)) if (precision + recall) != 0 else 0
    return f1_score



# funzione per il calcolo delle metriche per un certo insieme di immagini (test_img_list). 
# Calcola i veri positivi, falsi positivi e falsi negativi per ciascuna immagini, ne misura precision, recall, f1 per ognuna, e infine ne fa la media
def calculate_metrics(test_img_list, prediction_path, IOU_THRESH=0.5): 
    tot_true_positive = 0
    tot_false_positive = 0
    tot_false_negative = 0
    
    precision = 0
    recall = 0
    f1 = 0

    #prediction_path = '/kaggle/working/runs/detect/predict6/labels'
    label_path = '/kaggle/input/xview-dataset-team1/xview-dataset-team1/labels'
    
    # accede alle singole immagini sulle quali è stat fatta predizione
    for elem in test_img_list:
        
        true_positive = 0
        false_positive = 0
        false_negative = 0
        
        elem = elem.split('.')[0]
        elem = elem + '.txt'
        pred_img_path = os.path.join(prediction_path, elem)
        label_img_path = os.path.join(label_path, elem)
        
        if not os.path.exists(pred_img_path):     # se non esite il file delle predizioni, tutti i box della groud truth saranno falsi negativi
            with open(label_img_path, 'r') as file:
                false_negative = len(file.readlines())
                #print(f'TP:{true_positive}, FP:{false_positive}, FN:{false_negative}')
                
        else:   # se essite il file delle predizioni allora bisogna confrontare i box con quelli della ground truth
            
            with open(pred_img_path, 'r') as boxfile:
                boxes_pred = [line.strip() for line in boxfile]
                boxes_pred = [list(map(float, stringa.split())) for stringa in boxes_pred]
                boxes_pred = np.vstack(boxes_pred)
                
            with open(label_img_path, 'r') as boxfile:
                boxes_label = [line.strip() for line in boxfile]
                boxes_label = [list(map(float, stringa.split())) for stringa in boxes_label]
                if len(boxes_label) > 0:
                    boxes_label = np.vstack(boxes_label)
                
            change_coordinate_format(boxes_pred)
            change_coordinate_format(boxes_label)

            match_result = match_bboxes(boxes_label, boxes_pred, IOU_THRESH)   #treshold è il valore di iou oltre il quale c'è un match tra i box (di default è 0.7)
            #print(f'{match_result}\n')
            
            # calcolo falsi negativi e falsi positivi
            if boxes_label.shape[0] > boxes_pred.shape[0]:     # ho più box reali che predetti
                false_negative = boxes_label.shape[0] - boxes_pred.shape[0]
            elif boxes_label.shape[0] < boxes_pred.shape[0]:   # ho più box predetti che reali
                false_positive = boxes_pred.shape[0] - boxes_label.shape[0]
            
            for i in range(len(match_result[0])):
                #print(f'gt:{match_result[0][i]}, pred:{match_result[1][i]}, iou:{match_result[2][i]}, label:{match_result[3][i]}')
            
                index_gt = match_result[0][i]
                index_pred = match_result[1][i]
                if (boxes_label[index_gt,0] == boxes_pred[index_pred,0]) and (match_result[3][i] == 1):
                    true_positive += 1
                else:
                    false_positive += 1
                    false_negative += 1

            #print(f'TP:{true_positive}, FP:{false_positive}, FN:{false_negative}')
       
        # calcolo metriche
        precision += calculate_precision(true_positive, false_positive)
        recall += calculate_recall(true_positive, false_negative)
        f1 += calculate_f1_score(calculate_precision(true_positive, false_positive), calculate_recall(true_positive, false_negative))
    
        tot_true_positive += true_positive
        tot_false_positive += false_positive
        tot_false_negative += false_negative
        
    precision = precision/len(test_img_list)
    recall = recall/len(test_img_list)
    f1 = calculate_f1_score(precision, recall)
    f1 = f1/len(test_img_list)
    
    print(f'precision:{precision}, recall:{recall}, f1_score:{f1}')

## Training + Validation

Caricamento modello preaddestrato e fine tuning.  
Per l'addestramento della rete gli viene dato in input il file di configurazione <b>*xview_yolo.yaml*</b>, contentente le informazioni su dove recuperare le immagini di train e validation.

In [ ]:
# caricamento del modello preaddestrato
model = YOLO('yolov8s.pt')

# addestramento del modello sul dataset xview
results = model.train(data='/kaggle/input/xview-dataset-team1/xview-dataset-team1/YOLO_cfg/xview_yolo.yaml', epochs=100, imgsz=640, optimizer='Adam', lr0=0.001, name='train')

Caricamento del modello con i pesi migliori

In [ ]:
model_test = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')

Stampa delle loss e della mean average precision di train e validation

In [ ]:
print_plot('box_loss')
print_plot('cls_loss')
print_metric('mAP50(B)')

## Testing (labellato)
Procediamo ora a testare il modello su un set di immagini di test ricavate dal dataset di partenza, e delle quali conosciamo quindi le ground truth

#### Definiamo una serie di funzioni per dividere le immagini di test in tanti crop e rimettere insieme i risultati

Questa funzione calcola a partire da un'immagine le coordinate dei crop che dovranno essere realizzati.  
I crop possono essere realizzati con sovrapposizione per evitare di non riuscire a predirre oggetti troncati dai crop.

In [ ]:
def calculate_crop(original_width, original_heigth, scarto_sovrapposizione=200):
    #Definizione lista di coordinate per i crop delle immagini (lungo la direzione x)
    dim_x = original_width
    x_init = 0
    x_end = 640
    lista_x = []

    while (x_end <= dim_x):
        lista_x.append([x_init, x_end])
        x_init = x_end - scarto_sovrapposizione
        x_end = x_init + 640

    if (x_end > dim_x):
        lista_x.append([x_init, dim_x])

    #Definizione lista di coordinate per i crop delle immagini (lungo la direzione y)
    dim_y = original_heigth
    y_init = 0
    y_end = 640
    scarto_sovrapposizione = 200
    lista_y = []

    while (y_end <= dim_y):
        lista_y.append([y_init, y_end])
        y_init = y_end - scarto_sovrapposizione
        y_end = y_init + 640

    if (y_end > dim_y):
        lista_y.append([y_init, dim_y])
    
    return lista_x, lista_y

Questa funzione, a partire dalle liste recedentemente calcolate, esegue il crop delle immagini, salvandole nel path specificato (*crop_img_path*).  
Per assicurarsi che ciascun crop rispetti le dimensioni fissate di (640x640) viene effettuato un padding dove necessario.

In [ ]:
def create_crop(img, lista_x, lista_y, crop_img_path):
    lista_img = []

    for elem_y in lista_y:
        for elem_x in lista_x:
            dim_x = elem_x[1] - elem_x[0]
            dim_y = elem_y[1] - elem_y[0]
            if (dim_x == 640) and (dim_y == 640) :
                lista_img.append(img[elem_y[0]:elem_y[1], elem_x[0]:elem_x[1], :])
            else:
                temp = img[elem_y[0]:elem_y[1], elem_x[0]:elem_x[1], :]
                if (dim_x != 640):
                    padding = np.zeros((dim_y, 640-dim_x, 3), dtype=np.uint8)
                    temp = np.concatenate((temp,padding), axis=1)     #concatena l'immagine temp e gli zeri lungo la direzione orizzontale
                    dim_x = 640
                if (dim_y != 640):
                    padding = np.zeros((640-dim_y, dim_x, 3), dtype=np.uint8)
                    temp = np.concatenate((temp,padding), axis=0)     #concatena l'immagine temp e gli zeri lungo la direzione verticale
                    dim_y = 640
                lista_img.append(temp)

    # salva i crop nel percorso specificato
    for i, img_chunk in enumerate(lista_img):
        save_path = crop_img_path + 'image_' + str(i) + '.jpg'   #l'indice "i" serve per mantenere l'ordine dei crop
        plt.imsave(save_path, img_chunk)

Questa funzione può eventualmente essere usata per ricostruire l'immagine di partenza a partire dai suoi crop e dalle coordinate di ciascuno di essi

In [ ]:
def reconstruct_image(crop_img_path, posizioni):
    lista_img = os.listdir(crop_img_path)

    original_img = Image.new('RGB', (original_width, original_heigth), color='white')

    for img in os.listdir(crop_img_path):
        img_index = img.split('.')[0]
        img_index = int(img_index.split('_')[-1])
        img = Image.open(crop_img_path + img)
        original_img.paste(img, posizioni[img_index])

    original_img = np.array(original_img)
    plt.figure()
    plt.imshow(original_img)
    
    return original_img

Questa funzione permette di raggruppare le predizioni dei crop di una stessa immagine in un unico file, questo file sarà la predizione sull'immagine originale.

In [ ]:
def aggregate_label(crop_img_path, crop_label_path, output_file_path, posizioni, original_width, original_heigth):
    # qui va a prendere il crop creato, e ne calcola le dimensioni, questo serve per la denormalizzazione delle coordinate dei box
    for elem in os.listdir(crop_label_path):
        img_name = elem.split('.txt')[0]
        img_path = crop_img_path + img_name + '.jpg'
        img = plt.imread(img_path)
        heigth, width, band = img.shape

        with open(crop_label_path+elem, 'r') as file:
            boxes_label = [line.strip() for line in file]
            boxes_label = [list(map(float, stringa.split())) for stringa in boxes_label]

            # associa a ciascun box il crop sul quale è calcolato, quindi in base alla posizione del crop vengono adattate le coordinate
            index = int(img_name.split('_')[-1])
            with open(output_file_path, 'a') as file:
                for line in boxes_label:
                    # denormalizzazione coordinate
                    x_center = line[1] * width
                    y_center = line[2] * heigth
                    box_width = line[3] * width
                    box_heigth = line[4] * heigth
                    # shift + normalizzazione rispetto alle dimensioni dell'immagine originale
                    line[1] = (x_center + posizioni[index][0]) / original_width
                    line[2] = (y_center + posizioni[index][1]) / original_heigth
                    line[3] = box_width / original_width
                    line[4] = box_heigth / original_heigth

                    # scriviamo nel file contentente i box anche l'indice del crop dal quale proviene, ci servirà dopo per la rimozione dei box ridondanti
                    file.write(f"{line[0]} {line[1]} {line[2]} {line[3]} {line[4]} {index}\n")

Questa funzione ci permette di eliminare i box che risultano sovrapposti oltre una certa soglia.  
Questo si rende necessario dal momento che i crop sono eseguiti con sovrapposizione e ci sono quindi delle regioni comuni tra essi, che possono far si che uno stesso box sia predetto più volte.

In [ ]:
def remove_overlapped_boxes(label_path, treshold=0.25):
    indici_da_eliminare = []

    with open (label_path, 'r') as file:
        boxes = [line.strip() for line in file]
        boxes = [list(map(float, stringa.split())) for stringa in boxes]
        
        #print(boxes[0])

    # boxes_temp contiene i box con le coordinate convertite nel formato: (label,xmin,ymin,xmax,ymax)
    boxes_temp = copy.deepcopy(boxes)
    change_coordinate_format(boxes_temp)

    # alcuni box si sovrappongono perché i crop sono con sovrapposizione, quindi applichiamo una sorta di non maximal suppression
    # confrontiamo ciascun box con tutti gli altri della llista (escluso se stesso)
    # se due box appartenenti a crop diversi sono della stessa classe e il valore di sovrapposizione supera una certa soglia (calcolata con iou) allora il più piccolo dei due viene eliminato
    for i,box in tqdm(enumerate(boxes_temp)):
        for j,box_2 in enumerate(boxes_temp):
            if j != i:
                
                # confrontiamo i box da eliminare solo se provengono da crop diversi (index aggiunto prima), per eliminare solo i box dovuti alle sovrapposizioni dei crop (uso un basso valore di treshold)
                if box[5] != box_2[5]:      
                    
                    if box[0] == box_2[0]:
                        iou = bbox_iou(box[1:], box_2[1:])
                        if iou >= treshold:
                            area = (box[3] - box[1]) * (box[4] - box[2])    #area = (xmax-xmin)*(ymax-ymin)
                            area_2 = (box_2[3] - box_2[1]) * (box_2[4] - box_2[2])
                            if area_2 <= area:
                                indici_da_eliminare.append(j)
                                
                else:
                    # cerco di eliminare solo i box fortemente sovrapposti (treshold alto) dovuti ad una detection poco precisa del modello 
                    pass

    # la lista di indici è ordinata in modo decrescente, per evitare che durante l'eliminazione ci siano incongruenze tra gli indici
    # (eliminando in ordine crescente, il valore degli indici va a scalare e quindi non si trovano più)
    indici_da_eliminare = list(set(indici_da_eliminare))
    indici_da_eliminare.sort(reverse=True)
    for elem in indici_da_eliminare:
        del boxes[elem]

    # riscriviamo il file dei box,che conterrà ora solo i box non sovrapposti
    with open(label_path, 'w') as file:
        for line in boxes:
            file.write(f"{line[0]} {line[1]} {line[2]} {line[3]} {line[4]}\n")

Definizione della lista di immagini di test sulle quali testiamo il modello.  
Queste immagini sono a dimensione originale. Per fare detection il modello utilizzerà le funzioni precedentemente definite per creare i crop, fare previsioni su di essi, e rimettere poi insieme i risultati per ottenere la predizione sull'immagine di partenza.

In [ ]:
file_path = '/kaggle/input/xview-dataset-team1/xview-dataset-team1/YOLO_cfg/test.txt'

with open(file_path, 'r') as file:
    # Leggi tutte le righe del test.txt
    test_img_list = [line.strip() for line in file.readlines()]

for i,line in enumerate(test_img_list):
    test_img_list[i] = line.split('/')[-1]    # prendo il nome dell'immagine con l'estensione .jpg

In [ ]:
local_test_path = '/kaggle/input/xview-dataset-team1/xview-dataset-team1/images/'
local_crop_img_path = '/kaggle/working/local_crop_image/'
crop_pred_path = '/kaggle/working/runs/detect/predict/'
crop_label_path = '/kaggle/working/runs/detect/predict/labels/'
output_file_path = '/kaggle/working/local_test_predictions/'

if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)

if not os.path.exists(local_crop_img_path):
    os.mkdir(local_crop_img_path)
    
i=0

for elem in test_img_list:
    
    print(f'\ni:{i}')
    i += 1
    
    # apriamo l'immagine per ottenere la lunghezza e la larghezza
    img = plt.imread(local_test_path + elem)
    original_heigth, original_width, _ = img.shape
    # calcolo delle coordinate dei crop
    lista_x, lista_y = calculate_crop(original_width, original_heigth, scarto_sovrapposizione=200)
    
    # creazione di una directory che conterrà tutti i crop per l'immagine corrente
    crop_save_path = local_crop_img_path + elem.split('.')[0] + '/'
    if not os.path.exists(crop_save_path):
        os.mkdir(crop_save_path)
        # creazione dei crop per l'immagine corrente
        create_crop(img, lista_x, lista_y, crop_save_path)
    
    print(f'crop creati per {elem}')
    
    # predizione su tutti i crop dell'immagine corrente (sono contenuti in crop_save_path)
    temp = model_test.predict(crop_save_path, save_txt=True, verbose=False, name='predict')
    
    # viene creato un vettore di posizioni, che tiene conto della posizione di ciascun crop rispetto all'immagine originale (as es. (0,0), (0,640), (640,640), ...)
    # servono per adattare (ovvero traslare) i box calcolati sui crop all'immagine originale (quella grande)
    posizioni = []
    for elem_y in lista_y:
        for elem_x in lista_x:
            posizioni.append((elem_x[0],elem_y[0]))
            
    print(f'posizioni calcolate per {elem}')
    
    # i box dei diversi crop vengono uniti in un unico file, questi box vengono anche adattati all'immagine originale tramite traslazione e normalizzazione
    test_label_path = output_file_path + elem.split('.')[0] + '.txt'   # nome del file col quale sarà salvata la label di questa immagine
    aggregate_label(crop_save_path, crop_label_path, test_label_path, posizioni, original_width, original_heigth)
    print(f'label aggregate per {elem}')
    if os.path.exists(test_label_path):
        # vengono rimossi i box sovrapposti
        remove_overlapped_boxes(test_label_path, treshold=0.25)
        print(f'box rimossi per {elem}')
    
    
    shutil.rmtree(crop_pred_path)
    

Calcolo delle metriche di recall, precisione ed f1_score

In [ ]:
calculate_metrics(test_img_list, output_file_path, IOU_THRESH=0.5)

## Testing (non labellato)
Andiamo ora a testare il modello sul test set, quindi senza label.  

In [ ]:
test_path = '/kaggle/input/xview-dataset-team1/xview-dataset-team1/TestImages/'
crop_img_path = '/kaggle/working/crop_image/'
crop_pred_path = '/kaggle/working/runs/detect/predict/'
crop_label_path = '/kaggle/working/runs/detect/predict/labels/'
output_file_path = '/kaggle/working/test_predictions/'

if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)

if not os.path.exists(crop_img_path):
    os.mkdir(crop_img_path)
    
i=0
    
    
for elem in os.listdir(test_path):
    
    print(f'\ni:{i}')
    i += 1
    
    # apriamo l'immagine per ottenere la lunghezza e la larghezza
    img = plt.imread(test_path + elem)
    original_heigth, original_width, _ = img.shape
    lista_x, lista_y = calculate_crop(original_width, original_heigth)
    
    # creazione dei crop per la singola immagine
    crop_save_path = crop_img_path + elem.split('.')[0] + '/'
    if not os.path.exists(crop_save_path):
        os.mkdir(crop_save_path)
        create_crop(img, lista_x, lista_y, crop_save_path)
    
    print(f'crop creati per {elem}')
    
    temp = model_test.predict(crop_save_path, save_txt=True, verbose=False, name='predict')
    
    # viene creato un vettore di posizioni, che tiene conto della posizione di ciascun crop rispetto all'immagine originale (as es. (0,0), (0,640), (640,640), ...)
    # servono per adattare (ovvero spostare) i box calcolati sui crop all'immagine originale (quella grande)
    posizioni = []
    for y,elem_y in enumerate(lista_y):
        for elem_x in lista_x:
            posizioni.append((elem_x[0],elem_y[0]))
            
    print(f'posizioni calcolate per {elem}')
    
    # i box dei diversi crop vengono uniti in un unico file, questi box vengono anche adattati all'immagine originale tramite shift e normalizzazione
    test_label_path = output_file_path + elem.split('.')[0] + '.txt'
    aggregate_label(crop_save_path, crop_label_path, test_label_path, posizioni, original_width, original_heigth)
    print(f'label aggregate per {elem}')
    if os.path.exists(test_label_path):
        # vengono rimossi i box sovrapposti
        remove_overlapped_boxes(test_label_path, treshold=0.25)
        print(f'box rimossi per {elem}')
    
    
    shutil.rmtree(crop_pred_path)
    

#### Mostro i risultati su alcune immagini di test

In [ ]:
img = show_image_with_boxes('/kaggle/input/xview-dataset-team1/xview-dataset-team1/TestImages/367.jpg', '/kaggle/working/test_predictions/367.txt')
#img.save('/kaggle/working/test_img.jpg')

In [ ]:
img = show_image_with_boxes('/kaggle/input/xview-dataset-team1/xview-dataset-team1/TestImages/1464.jpg', '/kaggle/working/test_predictions/1464.txt')
#img.save('/kaggle/working/test_img.jpg')

In [ ]:
img = show_image_with_boxes('/kaggle/input/xview-dataset-team1/xview-dataset-team1/TestImages/2077.jpg', '/kaggle/working/test_predictions/2077.txt')
#img.save('/kaggle/working/test_img.jpg')

In [ ]:
img = show_image_with_boxes('/kaggle/input/xview-dataset-team1/xview-dataset-team1/TestImages/178.jpg', '/kaggle/working/test_predictions/178.txt')
#img.save('/kaggle/working/test_img.jpg')

In [ ]:
#shutil.rmtree('/kaggle/working/local_test_predictions')
#shutil.rmtree('/kaggle/working/local_crop_image')
#shutil.rmtree('/kaggle/working/test_predictions')
#shutil.rmtree('/kaggle/working/crop_image')
#shutil.rmtree('/kaggle/working/runs/detect/predict')